In [20]:
import tushare as ts
ts.set_token('8df6b825b87018ded5c09457e749cd1430d60febc9ab45d699eb0aab')
pro = ts.pro_api()

In [21]:
# Start get stock info
stockBasic = pro.query('stock_basic')
print(stockBasic)

        ts_code  symbol      name area industry market list_date
0     000001.SZ  000001      平安银行   深圳       银行     主板  19910403
1     000002.SZ  000002       万科A   深圳     全国地产     主板  19910129
2     000004.SZ  000004      国华网安   深圳      互联网     主板  19910114
3     000005.SZ  000005      世纪星源   深圳     环境保护     主板  19901210
4     000006.SZ  000006      深振业A   深圳     区域地产     主板  19920427
...         ...     ...       ...  ...      ...    ...       ...
4157  688777.SH  688777      中控技术   浙江     软件服务    科创板  20201124
4158  688788.SH  688788      科思科技   深圳     通信设备    科创板  20201022
4159  688819.SH  688819      天能股份   浙江     电气设备    科创板  20210118
4160  688981.SH  688981    中芯国际-U   上海      半导体    科创板  20200716
4161  689009.SH  689009  九号公司-UWD   北京     专用机械    CDR  20201029

[4162 rows x 7 columns]


In [22]:
#write into DB
from pyspark.sql import SparkSession
from pyspark.sql import SQLContext
spark = SparkSession.builder.appName("insertData").config("hive.metastore.uris", "thrift://localhost:9083").enableHiveSupport().getOrCreate()
stockBasicDF = spark.createDataFrame(stockBasic)
stockBasicDF.createOrReplaceTempView("stock_basic_tmp")
# spark.sql("create database sillytrader")
# spark.sql("create table sillytrader.stock_basic as select * from stock_basic_tmp")

DataFrame[]